In [1]:
import requests
import csv
from typing import List
import re
import pandas as pd
import sys

In [2]:
# access_token = ""
url = "https://api.hubapi.com/crm/v3/objects/contacts/search"
all_contacts = []
has_more = True
after = None

headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
}

properties = [
    "firstname", "lastname", "country", "phone", "training___create_date",
    "industry", "address", "checkbox", "raw_email_id", "hs_object_id"
]

data = {
    "filterGroups": [
        {
            "filters": [
                {
                    "propertyName": "allowed_to_collect",
                    "operator": "EQ",
                    "value": "true"
                }
            ]
        }
    ],
    "properties": properties,
    "limit": 100,
    "after": 0
}

In [4]:
while has_more:
    if after:
        data["after"] = after
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    result = response.json()
    for contact in result.get("results", []):
        row = {prop: contact["properties"].get(prop, "") for prop in properties}
        all_contacts.append(row)
    after = result.get("paging", {}).get("next", {}).get("after")
    has_more = after is not None

file_path = "../Challenge2/contact_collection.csv"
with open(file_path, mode="w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=properties)
    writer.writeheader()
    writer.writerows(all_contacts)